# Building Choropleth Maps with GeoPandas
*Mapping Quantities with Color Using Spatial Joins and Data Aggregation*


So far, we've worked a lot with spatial joins—but we haven't yet created one of the most sought-after visualizations in geospatial analysis: the **choropleth**.

A **choropleth** is a type of map where numerical values are represented by varying color shades. Lighter shades represent smaller values; darker shades represent larger ones. They're useful for visualizing things like:
- Population
- Crime rates
- Number of power plants
- Yelp scores
- Goat ownership (seriously)

Let's learn how to build one.


## Step 1: Get the Joined Data
We've already performed a spatial join where each power plant has been associated with the state it's located in.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Sample loading (replace with actual paths)
states_url = '../../geopandas_101_DATA/us/cb_2022_us_state_500k.zip'
plants_url = "../../geopandas_101_DATA/us/Power_Plants_in_the_US_5860152164617264051.geojson"

states = gpd.read_file(states_url)
plants = gpd.read_file(plants_url)

states.to_crs(plants.crs, inplace=True)
joined = gpd.sjoin(plants, states, how='inner', predicate='within')


## Step 2: Count Power Plants Per State
We now want to count how many power plants each state contains. This is a classic `.value_counts()` task.


In [ ]:
plant_counts = joined['NAME'].value_counts()
plant_counts.head()


At this point, we have what we need to build a choropleth—almost. We have state names and their associated power plant counts, but we’re missing the **geometry** to draw them.

That means we can’t map yet. So what’s the fix?



You might think, "Let me switch the spatial join and make states the first argument." That way, each row will include **state geometry**, which is what we want to plot.


In [ ]:
plants.sindex.valid_query_predicates

In [ ]:
# Get state geometry instead of plant points
states_with_plants = gpd.sjoin(states, plants, how='inner', predicate='contains')


Looks good—each row now contains a **state polygon** and plant data.

BUT there's a catch: if we inspect the shape of the resulting GeoDataFrame...


In [ ]:
print(states_with_plants.shape)


Uh-oh. We now have thousands of rows—one for **each power plant**, each including the same repeated state geometry.

We don’t want thousands of polygons. We want **one polygon per state**.

Time for a better approach.



## Step 3: Return to the Clean States GeoDataFrame
Remember that beautiful, clean `states` GeoDataFrame we started with? It has one shape per state. That’s what we want.

Let’s go back to that and find a way to merge in the plant counts.


In [ ]:
states = states.set_index('NAME')


Now that state names are our index, and `plant_counts` is also indexed by state name, we can **assign the values directly**.


In [ ]:
states['power_plant_count'] = plant_counts
states = states.sort_values(['power_plant_count'], 
                            ascending=False)
# states = states.reset_index()
states.head()

## Step 4: Plot the Choropleth

In [ ]:
def states_mapping(variable, legend_label):
    fig, ax = plt.subplots(figsize=(12, 10),)
    
    states.plot(
        ax=ax,
        column=variable, 
        edgecolor='gray',
        cmap='OrRd',
        legend=True, 
        legend_kwds={
            'label': legend_label,
            'orientation': 'horizontal',
            'shrink': 0.5
        }
    )
    
    # for idx, row in states.iterrows():
    #     centroid = row['geometry'].centroid
    #     ax.text(
    #         centroid.x, centroid.y, row['STUSPS'],
    #         fontsize=8, ha='center', va='center',
    #         color='black'
    #     )

    for centroid, label in zip(states['geometry'].centroid, states["STUSPS"]):
        ax.text(
            centroid.x, centroid.y, label, 
            fontsize=5, color='black', 
            ha='center', va='center',
        )

    
    ax.set_aspect('equal')
    
    # ax.axis("off")
    
    minx, miny, maxx, maxy = states.total_bounds
    # ax.set_xlim(minx, maxx)
    # ax.set_ylim(miny, maxy)
    ax.set_xlim([-180, -60])  # Approximate longitude bounds for continental US + Alaska
    ax.set_ylim([20, 75])     # Approximate latitude bounds
    
    plt.tight_layout()
    
    fig.savefig(f'output/{variable}.png', dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
states_mapping('power_plant_count', 'Number of Power Plants')


## Bonus: Add Another Variable — Total Megawatts
Now that we know how to summarize, let’s try something more numerical: summing total megawatts by state.


In [ ]:
megawatts_by_state = joined.groupby('NAME')['Total_MW'].sum()
states['megawatts'] = megawatts_by_state

In [ ]:
states_mapping('megawatts', 'Total MW')

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(15, 10))

# Plot the states as filled polygons

# Plot the power plants as red points
plants.plot(
# plants[plants['Plant_Name']=="Winnetka"].plot(
    ax=ax,
    color='red',
    markersize=1,
    alpha=0.7,
    label='Power Plants'
)

states.plot(
    ax=ax,
    color='none',
    edgecolor='black',
    linewidth=0.3
)

# Add title and clean up plot
ax.set_title("U.S. Power Plants by Location", fontsize=18, fontweight='bold')
ax.set_axis_off()
ax.legend()

# Improve layout
plt.tight_layout()

fig.savefig('output/Winnetka.png', dpi=300, )

plt.show()



## 🧠 Deep Dive: How Did That Assignment Work?
You might be wondering how GeoPandas knew where to put the counts and megawatts.

**Answer: the index.**

When you assign a Series to a DataFrame, Pandas matches rows by index—NOT by row order.

We set the index of `states` to state names. And our summaries are also indexed by state names. So assignment works smoothly:
```python
states['some_column'] = some_series
```
...works **as long as the indexes match**.



## ✅ Recap

- Spatial joins can link points (e.g., power plants) to areas (e.g., states).
- Aggregating values per region (e.g., `value_counts`, `groupby().sum()`) gives you something to color.
- Use the clean geometry dataset (one shape per area) for mapping.
- Always match indexes before assigning columns.

🗺️ You now know how to build choropleths from raw point data in GeoPandas!
